Otro intento de entrenamiento con GPT2 y el corpus de emilio

In [1]:
!pip install transformers[torch] numpy torch huggingface_hub transformers sentencepiece datasets
from datasets import Dataset, DatasetDict, load_dataset
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

In [2]:
# Cargar el archivo JSON
archivo_json = "corpus_rawV5_emilio.json"
with open(archivo_json, "r", encoding="utf-8") as f:
    datos = json.load(f)

# Convertir a un Dataset de Hugging Face
dataset = Dataset.from_list(datos)
print(dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 4605
})


In [3]:
# Modelo y tokenizador preentrenado
nombre_modelo = "gpt2"  # Cambia esto al modelo que deseas usar
tokenizer = AutoTokenizer.from_pretrained(nombre_modelo)
# Asignar el token de padding al token de fin de secuencia
tokenizer.pad_token = tokenizer.eos_token

dataset_filtrado = dataset.filter(lambda x: len(x["input"]) + len(x["output"]) <= 1000)
print(dataset_filtrado)


# Agregar formato personalizado
def preparar_ejemplo(ejemplo):

    texto = f"Pregunta: {ejemplo['input']}\nRespuesta: {ejemplo['output']}"
    return tokenizer(
        texto,
        truncation=True,
        padding="max_length",
        max_length=1024
    )

# Tokenizar el dataset
dataset_tokenizado = dataset.map(preparar_ejemplo, batched=False)
print(dataset_tokenizado)

# Configurar un Data Collator para padding dinámico, ya que no todos los mensajes tienen la misma longitud
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                 mlm=False)

#cargar el modelo
model = AutoModelForCausalLM.from_pretrained(nombre_modelo)

#preparar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./chatbot_entrenamientos",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
     gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    save_steps=250,
    logging_steps=100,
    fp16=True,  # Usa FP16 si es compatible con tu GPU
    save_total_limit=2,# Mantén solo los 2 checkpoints más recientes
    report_to="none", #para que no pida inicios de sesión ni nada
    max_steps=1000
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Filter:   0%|          | 0/4605 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output'],
    num_rows: 4605
})


Map:   0%|          | 0/4605 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 4605
})


In [ ]:
# Definir el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenizado, #_tokenizado,  # Usa el dataset tokenizado
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Entrenar el modelo
trainer.train() #aquí igual como argumento meter lo de: resume_from_checkpoint=True
#guardar lo que salga
model.save_pretrained("./chatbot_final")
tokenizer.save_pretrained("./chatbot_final")

<ipython-input-4-9d13484b18f0>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


Pra probar al final

In [ ]:
# Cargar el modelo entrenado
model = AutoModelForCausalLM.from_pretrained("./chatbot_final")
tokenizer = AutoTokenizer.from_pretrained("./chatbot_final")

# Generar texto
def generar_respuesta(prompt):
    entrada = f"Pregunta: {prompt}\nRespuesta:"
    entrada_tokenizada = tokenizer(entrada, return_tensors="pt")
    salida_ids = model.generate(entrada_tokenizada["input_ids"], max_length=128, num_beams=5, early_stopping=True)
    return tokenizer.decode(salida_ids[0], skip_special_tokens=True)

# Probar con un ejemplo
respuesta = generar_respuesta("¿Cómo está el clima hoy?")
print(respuesta)